In [44]:
import torch
import torch.nn as nn
import numpy as np

from pathlib import Path

In [45]:
DEVICE = 'cuda'
CHARSET = " .,!abcdefghijklmnopqrstuvwxyz"

In [46]:
# Model (Simple to get started)

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.ln1 = nn.Sequential(
                                nn.Linear(50,100),
                                nn.ReLU(),                                
                                nn.Linear(100,500),
                                nn.ReLU(),
                                nn.Linear(500,500),
                                nn.ReLU(),
                                )

        self.gru = nn.GRU(500, 500, batch_first=False)
        self.ln2 = nn.Sequential(
                        nn.Linear(500, 150),
                        nn.ReLU(),
                        nn.Linear(150,len(CHARSET)),
                         )

    def forward(self, x):
        x = self.ln1(x)
        x = nn.functional.relu(self.gru(x)[0])
        x = self.ln2(x)
        x = torch.nn.functional.log_softmax(x, dim=2)
        return x

model = Net().to(DEVICE)


In [47]:
model.load_state_dict(torch.load(f"models/{str(8)}.pt"))

<All keys matched successfully>

In [48]:
# TEST
TEST = 1
TEST2 = 1

FOLDER = Path('data/LJSpeech-1.1/wavs')

path_numpy = FOLDER.parent / Path('numpy')

path_data = path_numpy / Path('data')
path_labels = path_numpy / Path('labels')

data = np.load(path_data / Path(str(TEST) + '.npy'))
data = torch.tensor(data[TEST2]).float()
data = torch.nn.functional.normalize(data)
data = data[:,:,None].permute(1,2,0).to(DEVICE)
print(data.shape)

# Run the model
output = model(data)
output = output[:, 0, :].argmax(dim=1)
#print(output[0:800])
estimate = []
for char in output:
    if char != 0:
        estimate.append(CHARSET[char])
print(''.join(estimate))

torch.Size([1114, 1, 50])
ui


In [49]:
# VERIFY
labels = np.load(path_labels / Path(str(TEST) + '.npy'))

correct = []
for char in labels[TEST2]:
    if char != 0:
        correct.append(CHARSET[char-1])
print(''.join(correct))

twice a week there was a wine and beer club held at night, which lasted till two or three in the morning.
